In [45]:
import numpy as np
import csv

In [46]:
M=[]
movie = dict()
ratings = []
with open('MovieRatings.csv') as csv_file: # reading the movie files here
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    user_count=0
    user_past = 1
    for row in csv_reader:
        if(line_count!=0):

            key = int(row[1])
            user = int(row[0])
            rating = float(row[2])
            ratings.append(rating)
            if(user_past==user):
                user_count+=1
            else:
                user_count=1
                user_past = user
            if(user_count<=10):
                if key not in movie.keys():
                    movie[key] = [[rating,user]]
                else:
                    movie[key].append([rating,user])
            line_count+=1
        else:
            line_count+=1
ratings = np.array(ratings)

keyCount=0
for key in sorted(movie.keys()):
        #print(movie[key])
        keyCount+=1
        temp = np.zeros((100))
        for i in range(len(movie[key]) ):
            temp[movie[key][i][1]-1] = movie[key][i][0]
        M.append(temp)



M = np.array(M)
M = np.transpose(M)
print("No of Movies",M.shape[1])
print("No of users",M.shape[0])

    


No of Movies 317
No of users 100


In [47]:
movieNames = dict() # getting the movie names from the files
with open('movies.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count=0
    for row in csv_reader:
        if line_count!=0:
            name = row[1]
            genre = row[2]
            Id = int(row[0])
            movieNames[Id] = [name,genre]
            line_count+=1
        else:
            line_count+=1
    

count = 0
print("Some Movie names")
for key in movieNames.keys():
    print(movieNames[key][0])
    count+=1
    if(count==10):
        break






Some Movie names
Toy Story (1995)
Jumanji (1995)
Grumpier Old Men (1995)
Waiting to Exhale (1995)
Father of the Bride Part II (1995)
Heat (1995)
Sabrina (1995)
Tom and Huck (1995)
Sudden Death (1995)
GoldenEye (1995)


Ratings of 2 users on all the movies. As you can see the matrix is very sparse

In [48]:
for i in range(2):
    for j in range(317):
        print(M[i][j],end=" ")
    print("\n")

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 5.0 3.5 5.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 5.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 3.5 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 4.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 3.5 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 3.5 0.0 0.0 0.0 0.0 0.0 0.0 5.0 0.0 0.0 0.0 0.0 4.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

In [49]:

def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02): # edit the steps to 500 to produce quick results but will produce bad accuracy
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    
                    P[i] = P[i] + alpha * (2 * eij * Q[:,j] - beta * P[i])
                    Q[:,j] = Q[:,j] + alpha * (2 * eij * P[i] - beta * Q[:,j])

        eR = np.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:

            break
        if ( step % float(steps/100.0) ==0):
            print("Done",step/(steps/100))

    return P, Q.T

In [50]:


"""R = [

     [5,3,0,1,3],

     [4,0,0,1,3],

     [1,1,0,5,3],

     [1,0,0,4,0],

     [0,1,5,4,3],
    
     [2,1,3,0,3],

    ]

R = np.array(R)"""
# N: num of User
R = M
N = len(R)
# M: num of Movie
M = len(R[0])


# Num of Features
K = 10

 
P = np.random.rand(N,K)
Q = np.random.rand(M,K)

 

nP, nQ = matrix_factorization(R, P, Q, K)

nR = np.dot(nP, nQ.T)

#print(nR)


Done 0.0
Done 1.0
Done 2.0
Done 3.0
Done 4.0
Done 5.0
Done 6.0
Done 7.0
Done 8.0
Done 9.0
Done 10.0
Done 11.0
Done 12.0
Done 13.0
Done 14.0
Done 15.0
Done 16.0
Done 17.0
Done 18.0
Done 19.0
Done 20.0
Done 21.0
Done 22.0
Done 23.0
Done 24.0
Done 25.0
Done 26.0
Done 27.0
Done 28.0
Done 29.0
Done 30.0
Done 31.0
Done 32.0
Done 33.0
Done 34.0
Done 35.0
Done 36.0
Done 37.0
Done 38.0
Done 39.0
Done 40.0
Done 41.0
Done 42.0
Done 43.0
Done 44.0
Done 45.0
Done 46.0
Done 47.0
Done 48.0
Done 49.0
Done 50.0
Done 51.0
Done 52.0
Done 53.0
Done 54.0
Done 55.0
Done 56.0
Done 57.0
Done 58.0
Done 59.0
Done 60.0
Done 61.0
Done 62.0
Done 63.0
Done 64.0
Done 65.0
Done 66.0
Done 67.0
Done 68.0
Done 69.0
Done 70.0
Done 71.0
Done 72.0
Done 73.0
Done 74.0
Done 75.0
Done 76.0
Done 77.0
Done 78.0
Done 79.0
Done 80.0
Done 81.0
Done 82.0
Done 83.0
Done 84.0
Done 85.0
Done 86.0
Done 87.0
Done 88.0
Done 89.0
Done 90.0
Done 91.0
Done 92.0
Done 93.0
Done 94.0
Done 95.0
Done 96.0
Done 97.0
Done 98.0
Done 99.0


In [51]:
def take_first(elem):
    return elem[0]

curnt = 0
print("Predicted and actual ratings for already rated movies by user 1")
for key in sorted(movie.keys()):
    if R[0][curnt] !=0:
        if nR[0][curnt] >5:
            print(round(nR[0][curnt])," "," ",R[0][curnt],movieNames[key],"     Highly rated")
        else:
            if nR[0][curnt]>4.5:
                print(round(nR[0][curnt],2)," "," ",R[0][curnt],movieNames[key],"       Highly rated")
            else:
                print(round(nR[0][curnt],2)," "," ",R[0][curnt],movieNames[key])
            
        curnt+=1
    else:
        curnt+=1

print("\n\n")
curnt=0
num=0
temp = []
for key in sorted(movie.keys()):
    if nR[0][curnt]>4.5 and R[0][curnt]==0:
        num+=1
        temp.append([nR[0][curnt],movieNames[key]])
    curnt+=1

print("Recommended Movies to watch for user 1")

temp = sorted(temp,key=take_first,reverse=True)

for i in range(len(temp)):
    if (i<10):
        print(temp[i][1])

Predicted and actual ratings for already rated movies by user 1
4.93     5.0 ['Pulp Fiction (1994)', 'Comedy|Crime|Drama|Thriller']        Highly rated
3.49     3.5 ['Three Colors: Red (Trois couleurs: Rouge) (1994)', 'Drama']
5     5.0 ['Three Colors: Blue (Trois couleurs: Bleu) (1993)', 'Drama']      Highly rated
5     5.0 ['Underground (1995)', 'Comedy|Drama|War']      Highly rated
3.49     3.5 ["Singin' in the Rain (1952)", 'Comedy|Musical|Romance']
3.99     4.0 ['Dirty Dancing (1987)', 'Drama|Musical|Romance']
3.49     3.5 ['Delicatessen (1991)', 'Comedy|Drama|Romance']
3.49     3.5 ['Ran (1985)', 'Drama|War']
4.98     5.0 ['Seventh Seal, The (Sjunde inseglet, Det) (1957)', 'Drama']        Highly rated
3.99     4.0 ['Bridge on the River Kwai, The (1957)', 'Adventure|Drama|War']



Recommended Movies to watch for user 1
['Freeway (1996)', 'Comedy|Crime|Drama|Thriller']
['Reality Bites (1994)', 'Comedy|Drama|Romance']
['Dead Man Walking (1995)', 'Crime|Drama']
['8 1/2 (8½) (1963)', 